In [2]:
#%config IPCompleter.use_jedi=False
%load_ext autoreload
%autoreload 2

In [3]:
import sys 
import pandas as pd
import numpy as np

from pandas._libs.tslibs.timestamps import Timestamp
from pandas.core.frame import DataFrame
from pandas.core.series import Series

sys.path.append('..')
from cyclingEfficiency import Reader, Paths


In [7]:
def search_weight(
    date_to_search: Timestamp, df_weight: DataFrame, before: bool = False
) -> float:
    result: float = np.nan
    if before:
        df_result: DataFrame = df_weight[
            df_weight['date']<=date_to_search
        ]
        if df_result.shape[0] > 0:
            result = df_result.iloc[-1,1]    
    else: 
        df_result: DataFrame = df_weight[
            df_weight['date']>=date_to_search
        ]
        result = df_result.iloc[-1,1]
    return result

data =  Reader().data

date_min: Timestamp = data['activities']['date'].min()
date_max: Timestamp = data['activities']['date'].max()

weight: DataFrame = pd.DataFrame(
    pd.date_range(
        start=date_min,
        end=date_max,
        freq='d'
    ),
    columns=['date']
)

weight['weight'] = weight['date'].apply(
    search_weight, df_weight=data['weight'], before=True
)

In [8]:
df = pd.merge(
    left=data['activities'],
    right=weight,
    on='date',
    how='left'
)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3499537 entries, 0 to 3499536
Data columns (total 26 columns):
 #   Column     Dtype         
---  ------     -----         
 0   secs       int64         
 1   cad        float64       
 2   hr         float64       
 3   km         float64       
 4   kph        float64       
 5   nm         int64         
 6   watts      float64       
 7   alt        float64       
 8   lon        float64       
 9   lat        float64       
 10  headwind   int64         
 11  slope      float64       
 12  temp       float64       
 13  interval   int64         
 14  lrbalance  int64         
 15  lte        int64         
 16  rte        int64         
 17  lps        int64         
 18  rps        int64         
 19  smo2       int64         
 20  thb        int64         
 21  o2hb       int64         
 22  hhb        int64         
 23  datetime   datetime64[ns]
 24  date       datetime64[ns]
 25  weight     float64       
dtypes: datetime64[

In [15]:
df_filtered = df[
    (df['kph'] > 0) &
    (df['cad'] > 0) &
    (df['hr'] > 0) &
    (df['slope'] >= 0) &
    (~df['weight'].isna())
].reset_index()

In [16]:
df_filtered.shape

(657637, 27)

In [17]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657637 entries, 0 to 657636
Data columns (total 27 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   index      657637 non-null  int64         
 1   secs       657637 non-null  int64         
 2   cad        657637 non-null  float64       
 3   hr         657637 non-null  float64       
 4   km         657637 non-null  float64       
 5   kph        657637 non-null  float64       
 6   nm         657637 non-null  int64         
 7   watts      657637 non-null  float64       
 8   alt        657637 non-null  float64       
 9   lon        657637 non-null  float64       
 10  lat        657637 non-null  float64       
 11  headwind   657637 non-null  int64         
 12  slope      657637 non-null  float64       
 13  temp       657637 non-null  float64       
 14  interval   657637 non-null  int64         
 15  lrbalance  657637 non-null  int64         
 16  lte        657637 no